## Estimate Trip arrival times

In [1]:
import pandas as pd

vehicle_positions = pd.read_csv("data/vehicle_tracking/processed/804_lametro-rail/2019-01-16.csv", index_col=0, parse_dates=["datetime"])
stations = pd.read_csv("data/line_info/804/804_directionA_stations.csv", index_col=0)

In [2]:
directionA = vehicle_positions[vehicle_positions['direction'] == 0]
trips = directionA[['datetime', 'trip_id', 'relative_position']].groupby(['trip_id'])
estimates = stations[['stop_id', 'relative_position']]
estimates['estimate'] = True

def create_estimated_arrival_times(trip_id, trip):
    trip['estimate'] = False # add field to indicate these are actual observations not estimates
    trip_est = estimates # copy values into loop variable
    trip_est['trip_id'] = trip_id
    combined = trip.append(trip_est)
    combined = combined.sort_values('relative_position')
    combined = combined.reset_index(drop=True)
    combined['previous_pos'] = combined.relative_position.shift() # shift vals to move adjacent position and date data into each row
    combined['next_pos'] = combined.relative_position.shift(-1)
    combined['previous_dt'] = combined.datetime.shift()
    combined['next_dt'] = combined.datetime.shift(-1)
    select = combined[combined['estimate']==True]
    select['weight'] = (select.relative_position - select.previous_pos) / (select.next_pos - select.previous_pos)
    select['time_interpolation'] = (select.next_dt - select.previous_dt) * select.weight
    select['datetime'] = select.previous_dt + select.time_interpolation
    select['datetime_round'] = pd.DatetimeIndex(select.datetime).round('S')
    select['stop_id'] = pd.to_numeric(select.stop_id, downcast='integer')
    return select

groups = [create_estimated_arrival_times(trip_id, trip) for trip_id, trip in trips]
"DONE"

/home/cts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/cts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/cts/venv/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the 

'DONE'

In [3]:
all_estimates = pd.concat(groups)
stop_estimates = all_estimates.groupby(['stop_id'])

## Assemble/parse raw predictions data

In [4]:
import json
import os
import pendulum
from helpers.datetimefs import DateTimeFS, construct_filename

agency = "lametro-rail"
now = pendulum.now("America/Los_Angeles")
today = "2019-01-16"
end_datetime = now

def process_frame(datetime, path_base):
    source_path = construct_filename(path_base, datetime, ".json")
    with open(source_path, "r") as infile:
        raw = json.load(infile)["predictions"]
    try:
        directionB = raw[0]["direction"]["prediction"]
    except:
        return None
    if type(directionB) is dict:
        df = pd.DataFrame(directionB, index=[0])
    elif type(directionB) is list:
        df = pd.DataFrame(directionB[0], index=[0]) # get first prediction only
    df["datetime_prediction_made"] = pd.to_datetime(datetime.to_iso8601_string())
    df["datetime_predicted"] = pd.DatetimeIndex(pd.to_datetime(pd.to_datetime(df["epochTime"], unit='ms')))
    return df

line_predictions = {}
for line in range(804, 805):
    schedule_path = f"data/schedule/{line}_{agency}/{today}.csv"
    schedule = pd.read_csv(schedule_path, index_col=0)
    start_datetime = pendulum.parse(
        schedule.datetime.min(), tz="America/Los_Angeles"
    )  
    stops = stations.stop_id
    stop_predictions = {}
    for stop_id in stops:
        path_base = f"data/predictions/raw/{line}_{agency}/{stop_id}"
        dtfs = DateTimeFS(path_base)
        datetimes = dtfs.get_datetimes_in_range(
            start_datetime.in_tz("UTC"), end_datetime.in_tz("UTC")
        )
        predictions = [process_frame(datetime, path_base) for datetime in datetimes]
        predictions = pd.concat(predictions).drop_duplicates(["dirTag", "datetime_predicted", "vehicle"])[["datetime_prediction_made", "datetime_predicted", "vehicle", "tripTag"]]
        stop_predictions[stop_id] = predictions
    line_predictions[line] = pd.concat(stop_predictions)
all_predictions = pd.concat(line_predictions)
"DONE"

/home/cts/venv/lib/python3.6/site-packages/ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



'DONE'

In [6]:
os.makedirs("data/predictions/processed/804_lametro-rail", exist_ok=True)
all_predictions.to_csv("data/predictions/processed/804_lametro-rail/2019-01-16.csv")

In [7]:
def match_predictions_and_schedule(key, group):
    stop_id = key[1]
    est = stop_estimates.get_group(stop_id).dropna(subset=['datetime'])
    est = est.set_index(pd.DatetimeIndex(est['datetime'])).sort_index()
    if len(est):
        group.loc[:,"closest_actual"] = group.datetime_predicted.apply(lambda x: est.index[est.index.get_loc(x, method='nearest')])
        group.loc[:,"scheduled"] = group.tripTag.apply(lambda x: schedule[(schedule["trip_id"] == int(x)) & (schedule["stop_id"] == int(stop_id))].datetime.values[0])
        return group
    return None

comparable = pd.concat([match_predictions_and_schedule(key, group) for key, group in all_predictions.groupby(level=[0, 1])])
comparable["difference"] = comparable["datetime_predicted"] - pd.DatetimeIndex(comparable["closest_actual"])
"DONE"

/home/cts/venv/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/cts/venv/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


'DONE'

In [10]:
os.makedirs("data/predictions/comparison/804_lametro-rail", exist_ok=True)
comparable.to_csv("data/predictions/comparison/804_lametro-rail/2019-01-16.csv")

In [4]:
import pandas as pd

comparable = pd.read_csv("data/predictions/comparison/804_lametro-rail/2019-01-16.csv", index_col=(0, 1, 2))

In [19]:
comparable["datetime_predicted_round"] = pd.DatetimeIndex(comparable.datetime_predicted).round('T')
comparable["datetime_predicted"] = pd.DatetimeIndex(comparable.datetime_predicted)
comparable = comparable.drop_duplicates(["vehicle", "tripTag", "datetime_predicted_round"])
"DONE"

'DONE'

In [20]:
comparable.head()

datetime_prediction_made      datetime_predicted  \
    Unnamed: 1 Unnamed: 2                                                    
804 80402      0           2019-01-16 12:04:10.000 2019-01-16 12:34:00.000   
               0           2019-01-16 12:31:22.000 2019-01-16 12:35:19.696   
               0           2019-01-16 12:35:09.000 2019-01-16 12:49:00.000   
               0           2019-01-16 12:49:07.000 2019-01-16 12:54:00.000   
               0           2019-01-16 12:53:09.000 2019-01-16 13:02:00.000   

                          vehicle   tripTag                 closest_actual  \
    Unnamed: 1 Unnamed: 2                                                    
804 80402      0              451  47674328  2019-01-16 12:33:25.300400676   
               0              451  47674328  2019-01-16 12:33:25.300400676   
               0              452  47674344  2019-01-16 12:48:17.506436539   
               0              402  47674229  2019-01-16 12:48:17.506436539   
               0              412  47674361  2019-01-16 13:01:25.716524116   

                                     scheduled      difference  \
    Unnamed: 1 Unnamed: 2                                        
804 80402      0           2019-01-16 04:34:00 00:00:34.699599   
               0           2019-01-16 04:34:00 00:01:34.699599   
               0           2019-01-16 04:49:00 00:00:42.493563   
               0           2019-01-16 04:54:00 00:05:42.493563   
               0           2019-01-16 05:02:00 00:00:34.283475   

                          datetime_predicted_round  
    Unnamed: 1 Unnamed: 2                           
804 80402      0               2019-01-16 12:34:00  
               0               2019-01-16 12:35:00  
               0               2019-01-16 12:49:00  
               0               2019-01-16 12:54:00  
               0               2019-01-16 13:02:00

In [26]:
comparable["difference"] = comparable["datetime_predicted_round"] - pd.DatetimeIndex(comparable["closest_actual"])
comparable.difference.head()

     Unnamed: 1  Unnamed: 2
804  80402       0            00:00:34.699599
                 0            00:01:54.395599
                 0            00:00:42.493563
                 0            00:05:42.493563
                 0            00:00:34.283475
Name: difference, dtype: timedelta64[ns]

## Results

Mean difference between predicted time and arrival time:

In [27]:
comparable.difference.mean()

Timedelta('-1 days +23:59:46.383734')

Standard deviation of predicted time and arrival time:

In [28]:
comparable.difference.std()

Timedelta('0 days 00:06:50.040637')

In [29]:
len(comparable)

4388